In [37]:
from dash import Dash, dcc, html, dash_table
import dash_leaflet as dl
import plotly.express as px
from dash.dependencies import Input, Output, State

import base64 
import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacuser"  
password = "Password123" 
shelter = AnimalShelter(username, password)

# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))
if "_id" in df.columns:
    df.drop(columns=['_id'], inplace=True)  # Avoid ObjectId issues in DataTable

#########################
# Dashboard Layout / View
#########################
app = Dash(__name__)

# Grazioso Salvare’s logo
image_filename = 'GraziosoSalvareLogo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    # html.Div(id='hidden-div', style={'display':'none'}),  # Optional hidden div
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), height='100px')),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.H3('By: Ami Akagi')),

    # Filter Options with Radio buttons
    html.Hr(),
    
    html.Div([
        html.Span('Filter: ', style={'fontWeight': 'bold', 'marginRight': '10px'}),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'All', 'value': 'all'},
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain Rescue', 'value': 'mountain'},
                {'label': 'Disaster Rescue', 'value': 'disaster'}
            ],
            value='all',
            inline=True
        )
    ], style={'display': 'flex', 'alignItems': 'center', 'marginBottom': '10px'}),

    html.Hr(),

    #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
    #If you completed the Module Six Assignment, you can copy in the code you created here 
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        filter_action='native',
        sort_action='native',
        row_selectable='single',
        selected_rows=[0],
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left'}
    ),

    html.Br(),

    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    [Output('datatable-id','data'),
     Output('datatable-id','columns')],
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    ### Filter interactive data table with MongoDB queries

    if filter_type == 'water':
        query = {"breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
                 "age_upon_outcome_in_weeks": {"$lte": 104}}
    elif filter_type == 'mountain':
        query = {"breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
                 "age_upon_outcome_in_weeks": {"$lte": 104}}
    elif filter_type == 'disaster':
        query = {"breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
                 "age_upon_outcome_in_weeks": {"$lte": 104}}
    else:
        query = {}

    dff = pd.DataFrame.from_records(shelter.read(query))
    if "_id" in dff.columns:
        dff.drop(columns=['_id'], inplace=True)

    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in dff.columns]
    return dff.to_dict('records'), columns


@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns or []]  # fallback for None


@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")]
)
def update_graphs(viewData):
    # No Data   
    if viewData is None or len(viewData) == 0:
        return [html.Div("No data to display")]
    # Pie Chart
    dff = pd.DataFrame(viewData)
    fig = px.pie(dff, names='breed', title='Breed Distribution')
    return [dcc.Graph(figure=fig)]


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")]
)
def update_map(viewData):
    # No Data
    if viewData is None or len(viewData) == 0:
        return [html.Div("No map data available.")] 

    dff = pd.DataFrame(viewData)
    if 'location_lat' not in dff or 'location_long' not in dff:
        return [html.Div("Location data missing.")]

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            *[
                dl.Marker(position=[row['location_lat'], row['location_long']], children=[
                    dl.Tooltip(row.get('breed', 'Unknown')),
                    dl.Popup([
                        html.H4("Animal Name"),
                        html.P(str(row.get('name', 'No name')))
                    ])
                ]) for _, row in dff.iterrows()
            ]
        ])
    ]

if __name__ == '__main__':
    app.run(debug=True)